Dưới đây là hướng dẫn chi tiết song ngữ (English-Vietnamese) cho chủ đề:

**🔬 Topic\_CheMBL\_35\_2: Predicting biological activity from molecular descriptors**
**🔍 Chủ đề: Dự đoán hoạt tính sinh học từ các đặc trưng phân tử**

---

## 🧪 1. Analytical Framework | Mô hình phân tích

**English**
We aim to predict the biological activity (e.g., IC₅₀) of molecules using molecular descriptors like Molecular Weight (MolWt), Topological Polar Surface Area (TPSA), LogP, etc., derived via RDKit.
This QSAR modeling task involves:

* Querying ChEMBL for assay data (standard\_value for IC50)
* Filtering and cleaning
* Generating molecular descriptors
* Training machine learning models (e.g., Random Forest)

**Tiếng Việt**
Mục tiêu là dự đoán hoạt tính sinh học (ví dụ IC₅₀) của các phân tử dựa vào đặc trưng phân tử như MolWt, TPSA, LogP… được tạo bởi RDKit.
Bài toán QSAR gồm:

* Truy vấn dữ liệu thí nghiệm từ ChEMBL (standard\_value cho IC50)
* Làm sạch dữ liệu
* Tính toán đặc trưng hóa học
* Huấn luyện mô hình học máy (ví dụ Random Forest)

---

## 📦 2. Folder Structure | Cấu trúc thư mục AIMLOps

```plaintext
project_root/
│
├── data/                ← Exported CSV from SQL (max 100 rows)
│   └── Topic_CheMBL_35_2_data.csv
│
├── notebook/
│   └── Topic_CheMBL_35_2_1_query_and_descriptors.ipynb
│   └── Topic_CheMBL_35_2_2_model_training.ipynb
│
├── src/
│   └── descriptors.py   ← Functions to calculate descriptors
│
└── scripts/
    └── run_model.py     ← Optional CLI for batch runs
```

---

## 🧾 3. Example SQL Query (limit 100 rows) | Ví dụ truy vấn SQL (giới hạn 100 dòng)

```sql
-- File: get_ic50_cleaned.sql
SELECT md.chembl_id,
       cs.canonical_smiles,
       act.standard_value::float AS ic50_nM
FROM activities act
JOIN compound_structures cs ON act.molregno = cs.molregno
JOIN molecule_dictionary md ON md.molregno = cs.molregno
WHERE act.standard_type = 'IC50'
  AND act.standard_value ~ '^[0-9\.]+$'
  AND act.standard_units = 'nM'
  AND act.standard_value::float < 100000
LIMIT 100;
```

> 💡 Lưu ý: Dùng `act.standard_value::float` thay vì `~` với kiểu numeric để tránh lỗi `operator does not exist`.

---

## 🧬 4. Python Script for Descriptor Calculation | Tính descriptor bằng Python

```python
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

df = pd.read_csv('../data/Topic_CheMBL_35_2_data.csv')
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)
df.drop(columns='mol', inplace=True)
df.head()
```

---

## 🤖 5. Train Random Forest Model (compatible with old scikit-learn)

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X = df[['MolWt', 'TPSA', 'LogP']]
y = df['ic50_nM']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("R2 score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)  # Compatible format
```

---

## 💡 6. Additional SQL & Python Examples | 5 ví dụ bổ sung

### 🔹 SQL 1: Select compounds with pChEMBL value

```sql
SELECT md.chembl_id, cs.canonical_smiles, act.pchembl_value
FROM activities act
JOIN molecule_dictionary md ON md.molregno = act.molregno
JOIN compound_structures cs ON cs.molregno = md.molregno
WHERE act.pchembl_value IS NOT NULL
LIMIT 100;
```

### 🔹 SQL 2: Fetch compounds active on targets

```sql
SELECT md.chembl_id, cs.canonical_smiles, tgt.pref_name
FROM activities act
JOIN compound_structures cs ON act.molregno = cs.molregno
JOIN molecule_dictionary md ON md.molregno = cs.molregno
JOIN target_dictionary tgt ON act.target_id = tgt.tid
WHERE act.standard_type = 'IC50' AND act.standard_value ~ '^[0-9\.]+$'
LIMIT 100;
```

### 🔹 Python 1: Add Num of H-Donors and Acceptors

```python
from rdkit.Chem import Lipinski
df['NumHDonors'] = df['mol'].apply(Lipinski.NumHDonors)
df['NumHAcceptors'] = df['mol'].apply(Lipinski.NumHAcceptors)
```

### 🔹 Python 2: Export processed dataset

```python
df.to_csv('../data/processed_descriptors.csv', index=False)
```

### 🔹 Python 3: Save model

```python
import joblib
joblib.dump(model, '../models/qsar_ic50_rf.pkl')
```

---

Nếu bạn muốn mình đóng gói lại thành 2 notebook chuẩn AIMLOps (`Topic_CheMBL_35_2_1_query_and_descriptors.ipynb`, `Topic_CheMBL_35_2_2_model_training.ipynb`) hoặc tạo ZIP dự án mẫu, vui lòng yêu cầu tiếp nhé.


In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

df = pd.read_csv('../data/Topic_CheMBL_35_2_data.csv')
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)
df.drop(columns='mol', inplace=True)
df.head()

,chembl_id,canonical_smiles,ic50_nm,MolWt,TPSA,LogP
0,CHEMBL324340,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,2500.0,398.374,100.71,4.30202
1,CHEMBL324340,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,50000.0,398.374,100.71,4.30202
2,CHEMBL109600,COc1ccccc1-c1ccc2oc(-c3ccc(OC)c(N4C(=O)c5ccc(C...,9000.0,520.497,119.17,5.67780
3,CHEMBL357278,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4ccc(Cl)c(C(...,4000.0,543.011,77.93,4.27292
4,CHEMBL357119,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)NCCc4ccccc4)CC...,17000.0,468.623,77.93,2.32092


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X = df[['MolWt', 'TPSA', 'LogP']]
y = df['ic50_nm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("R2 score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)  # Compatible format


R2 score: -0.6935868754184276
RMSE: 15040.909614321108
